In [1]:
python.exe -m pip install --upgrade pip
import numpy as np
import opendatasets as od
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from unidecode import unidecode

ModuleNotFoundError: No module named 'numpy'

In [ ]:
cadena_con_tildes = "Canción con acentos y tildes"
cadena_sin_tildes = unidecode(cadena_con_tildes)

print(cadena_sin_tildes)

In [ ]:
# od.download("https://www.kaggle.com/datasets/shivd24coder/fifa-world-cup-dataset")

In [ ]:
attendence_df = pd.read_csv(r'fifa-world-cup-dataset\attendence.csv')
goals_df = pd.read_csv(r'fifa-world-cup-dataset\Goals.csv')
match_df = pd.read_csv(r'fifa-world-cup-dataset\match.csv')
team_df = pd.read_csv(r'fifa-world-cup-dataset\Team.csv')

print('Attendence\n')
print(attendence_df.info(),2*'\n', 50*'*','\n')
print('Goals\n')
print(goals_df.info(), 2*'\n', 50*'*','\n')
print('Match\n')
print(match_df.info(), 2*'\n', 50*'*','\n')
print('Team\n')
print(team_df.info())

In [ ]:
attendence_df.head()

In [ ]:
goals_df.head()

In [ ]:
match_df.head()

In [ ]:
team_df.head()

In [ ]:
def convert_to_integer(value):
    if value.isdigit():
        return int(value)
    elif str(type(value)) in ['int', 'float']:
        return value
    return 0

def date(cadena):
    cadena = str(cadena)
    if cadena == 'nan':
        return np.nan
    patron_1 = '(\d{1,2})-(\d{1,2})-(\d{4}) \((\d{1,2}):(\d{1,2}) h\)'
    match_1 = re.search(patron_1, cadena)
    
    patron_2 = '(\d{1,2})-(\d{1,2})-(\d{4})'
    match_2 = re.search(patron_2, cadena)

    if match_1 is not None:
        fecha = pd.to_datetime(cadena.strip(), format='%d-%m-%Y (%H:%M h)')
    elif match_2 is not None:
        indice_inicial = cadena.find('(')
        indice_final = cadena.find(')')+1
        fecha = cadena.replace(cadena[indice_inicial:indice_final], '')
        fecha = pd.to_datetime(fecha.strip(), format='%d-%m-%Y')
    else:
        return np.nan
    return fecha

def string_and_strip(value):
    if value == 'nan':
        return np.nan
    else:
        return str(value).strip()

def encode_decode(cadena):
    if str(cadena) == 'nan':
        return np.nan
    else:
        cadena_corregida = str(cadena).encode('latin-1').decode('utf-8')
        cadena_corregida = unidecode(cadena_corregida)
        return cadena_corregida

def stage_string_correction(cadena):
    cadena_corregida = cadena.encode('iso-8859-1').decode('utf-8')
    cadena_corregida = cadena_corregida.replace('\r\n', '').strip()
    palabras = cadena_corregida.split()  # Divide la cadena en palabras
    cadena_corregida = ' '.join(palabras)  # Une las palabras con un solo espacio como separador
    return cadena_corregida

def extraer_texto_entre_parentesis(cadena):
    if type(cadena) == str:
        indice_inicial = cadena.find('(')
        indice_final = cadena.find(')')
        if indice_inicial >= 0:
            texto_entre_parentesis = cadena[indice_inicial + 1 :indice_final]
            return texto_entre_parentesis.strip()
        else:
            return np.nan
    else:
        return cadena
    
    
def extraer_texto_fuera_de_parentesis(cadena):
    if type(cadena) == str:
        indice_inicial = cadena.find('(')
        indice_final = cadena.find(')')
        if indice_inicial >= 0:
            texto_fuera_de_parentesis = cadena.replace(cadena[indice_inicial:indice_final + 1], '')
            return texto_fuera_de_parentesis.strip()
        else:
            return cadena.strip()
    else:
        return cadena


In [ ]:
attendence_df_work = attendence_df.copy()

In [ ]:
attendence_df_work['ATTENDANCE'] = attendence_df_work['ATTENDANCE'].apply(convert_to_integer)
attendence_df_work['DATE'] = attendence_df_work['DATE'].apply(date)
attendence_df_work['PK'] = attendence_df_work['PK'].apply(string_and_strip).apply(encode_decode)
attendence_df_work['REFEREE'] = attendence_df_work['REFEREE'].apply(encode_decode)
attendence_df_work['STADIUM'] = attendence_df_work['STADIUM'].apply(encode_decode)
attendence_df_work['stage'] = attendence_df_work['stage'].apply(stage_string_correction)
attendence_df_work['team1'] = attendence_df_work['team1'].apply(encode_decode)
attendence_df_work['team2'] = attendence_df_work['team2'].apply(encode_decode)

In [ ]:
attendence_df_work.nlargest(20, columns='ATTENDANCE')

In [ ]:
attendence_df_work.loc[attendence_df['PK'].notnull()]

In [ ]:
goals_df_work = goals_df.copy()
goals_df_work['player'] = goals_df_work['player'].apply(encode_decode)

In [ ]:
match_df_work = match_df.copy()

match_df_work['team1_name'] = match_df_work['team1'].apply(extraer_texto_entre_parentesis)
match_df_work['team2_name'] = match_df_work['team2'].apply(extraer_texto_entre_parentesis)

match_df_work['team1'] = match_df_work['team1'].apply(extraer_texto_fuera_de_parentesis)
match_df_work['team2'] = match_df_work['team2'].apply(extraer_texto_fuera_de_parentesis)

In [ ]:
match_df_work

In [ ]:
team_df_work = team_df.copy()

team_df_work['DOB_num'] = team_df_work['DOB/Age'].apply(extraer_texto_entre_parentesis)
team_df_work['DOB/Age'] = team_df_work['DOB/Age'].apply(extraer_texto_fuera_de_parentesis)
team_df_work['Age'] = team_df_work['DOB/Age'].apply(extraer_texto_entre_parentesis)
team_df_work['DOB/Age'] = team_df_work['DOB/Age'].apply(extraer_texto_fuera_de_parentesis)

In [ ]:
team_df_work 